# **Klasifikasi Kematangan Pisang**

Notebook ini mendemonstrasikan proses klasifikasi tingkat kematangan pisang (mentah, matang, busuk) berdasarkan data citra RGB. Proses ini mencakup pemuatan data, *feature engineering*, pelatihan model, dan evaluasi menggunakan CatBoost.

In [2]:
# pip install -q catboost pandas scikit-learn matplotlib
import pandas as pd
import numpy as np

df = pd.read_csv("hasil_rgb_pisang (1).csv")  # kolom: No, R, G, B, Kategori
print(df.head())
print(df.groupby("Kategori").apply(lambda x: x.sample(5, random_state=42)))
print(df['Kategori'].value_counts())


   No    R    G    B Kategori
0   1  129  131  119   matang
1   2  133  134  107   matang
2   3  146  149  137   matang
3   4  183  188  174   matang
4   5  226  226  214   matang
                 No    R    G    B Kategori
Kategori                                   
busuk    609    610  138  130   99    busuk
         680    681  148  142  136    busuk
         433    434  143  138  119    busuk
         610    611   88   82   67    busuk
         493    494   80   80   80    busuk
matang   209    210  128  133  124   matang
         280    281  109  111   99   matang
         33      34  175  179  163   matang
         210    211  159  159  142   matang
         93      94  132  129  110   matang
mentah   1009  1010  191  203  162   mentah
         1080  1081  180  183  158   mentah
         833    834  146  152  136   mentah
         1010  1011  146  156  121   mentah
         893    894  193  198  163   mentah
Kategori
matang    400
busuk     400
mentah    400
Name: count, dtype: i

/tmp/ipython-input-4218356931.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  print(df.groupby("Kategori").apply(lambda x: x.sample(5, random_state=42)))


In [3]:
import colorsys

EPS = 1e-6

def add_features(d):
    d = d.copy()
    # pastikan float
    for c in ['R','G','B']:
        d[c] = d[c].astype(float)

    s = d[['R','G','B']].sum(axis=1) + EPS
    d['brightness'] = s / 3.0
    d['r_chroma'] = d['R'] / s
    d['g_chroma'] = d['G'] / s
    d['b_chroma'] = d['B'] / s
    d['rg_ratio'] = d['R'] / (d['G'] + EPS)
    d['gb_ratio'] = d['G'] / (d['B'] + EPS)
    d['br_ratio'] = d['B'] / (d['R'] + EPS)
    hsv_list = d[['R','G','B']].apply(
        lambda row: colorsys.rgb_to_hsv(row['R']/255, row['G']/255, row['B']/255), axis=1
    )
    d['hue'] = [h for h,s,v in hsv_list]
    d['saturation'] = [s for h,s,v in hsv_list]
    d['value'] = [v for h,s,v in hsv_list]
    return d

df_fe = add_features(df)
feature_cols = ['R','G','B','brightness','r_chroma','g_chroma','b_chroma','rg_ratio','gb_ratio','br_ratio']
X = df_fe[feature_cols]
y = df_fe['Kategori']


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
print(y_train.value_counts(normalize=True))


Kategori
matang    0.333333
mentah    0.333333
busuk     0.333333
Name: proportion, dtype: float64


In [5]:
pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [6]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='TotalF1',          # atau 'Accuracy'
    iterations=1000,                # mulai 1000; nanti bisa turunkan/naikkan
    depth=6,                        # kedalaman tree
    learning_rate=0.1,              # step size
    l2_leaf_reg=3.0,                # regularization
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50,
    auto_class_weights='Balanced'   # aktifkan kalau distribusi label tidak seimbang
)

model.fit(X_train, y_train, eval_set=(X_test, y_test))


0:	learn: 0.7899372	test: 0.8281013	best: 0.8281013 (0)	total: 79.7ms	remaining: 1m 19s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8684903332
bestIteration = 29

Shrink model to first 30 iterations.


In [7]:
try:
    model.save_model("catboost_banana_model.cbm")
    print("Model CatBoost berhasil disimpan ke 'catboost_banana_model.cbm'")
except Exception as e:
    print(f"Gagal menyimpan model: {e}")

Model CatBoost berhasil disimpan ke 'catboost_banana_model.cbm'
